In [1]:
import ee
import os, shutil
import rasterio
import numpy as np
from pprint import pprint
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
from channel_classifier.internal.ee_utils import add_labels, add_cloud_score, add_cloud_label, remap_modis_labels, get_landsat_image
from channel_classifier.internal.utils import download_tiff, read_image_list

In [3]:
import tempfile, urllib, zipfile
def download_tiff(image, filename=None, scale=100):
    if not filename:
        filename = tempfile.mkdtemp()

    url = ee.data.makeDownloadUrl(ee.data.getDownloadId({
        'image': image.serialize(),
        'scale': '{:d}'.format(scale),
        'filePerBand': 'false',
        'name': 'data',
    }))
    local_zip, headers = urllib.urlretrieve(url)
    with zipfile.ZipFile(local_zip) as local_zipfile:
        local_zipfile.extract('data.tif', '')
        shutil.copyfile('data.tif', filename)
    return filename

In [4]:
ee.Initialize()

In [5]:
datasets = {'LE7' : 'LANDSAT/LE07/C01/T1_TOA',
            'LT5' : 'LANDSAT/LT05/C01/T1_TOA',
           }

In [6]:
image_list = read_image_list('imagelist.txt')

In [7]:
thresh = 50
filenames = []
for sat, path, row, date in image_list[0:100]:
    dataset_path = datasets[sat]
    year, month, day = map(int, date.split('-'))
    filename = 'images/{}_{}{:02d}{:02d}_{:03d}{:03d}.tif'.format(sat, year, month, day, path, row)
    
    #try:
    im = get_landsat_image(path, row, year, month, day, dataset_path=dataset_path)
    
    if im.getInfo() is not None:
        cc = im.get('CLOUD_COVER').getInfo()
        if cc < thresh:
            im = add_labels(im)
            im = remap_modis_labels(im)
            im = add_cloud_score(im)
            im = add_cloud_label(im)
            download_tiff(im, filename, scale=240)
            print('Downloaded ' + filename + ' (cc: {:d}%)'.format(cc))
        else:
            print('Image ' + filename + ' is too cloudy (cc: {:d}%)'.format(cc))
            
    #except:
    #    print('Image {} failed'.format(filename))

Downloaded images/LE7_20120402_007063.tif (cc: 31.0%)
Downloaded images/LE7_20120402_007064.tif (cc: 31.0%)
Image images/LE7_20090403_006064.tif is too cloudy (cc: 85.0%)
Image images/LE7_20010404_007063.tif is too cloudy (cc: 78.0%)
Downloaded images/LE7_20010404_007064.tif (cc: 35.0%)
Image images/LE7_20150404_006064.tif is too cloudy (cc: 57.0%)
Image images/LE7_20130405_007063.tif is too cloudy (cc: 62.0%)
Image images/LE7_20020407_007064.tif is too cloudy (cc: 83.0%)
Downloaded images/LE7_20080407_007063.tif (cc: 7.0%)
Downloaded images/LE7_20080407_007064.tif (cc: 39.0%)
Image images/LE7_20140408_007064.tif is too cloudy (cc: 83.0%)
Image images/LE7_20000410_006064.tif is too cloudy (cc: 58.0%)
Image images/LE7_20030410_007064.tif is too cloudy (cc: 76.0%)
Image images/LE7_20060411_006064.tif is too cloudy (cc: 63.0%)
Downloaded images/LE7_20040412_007064.tif (cc: 37.0%)
Downloaded images/LE7_20100413_007063.tif (cc: 47.0%)
Image images/LE7_20100413_007064.tif is too cloudy (cc: 

EEException: Image.load: Image asset 'MODIS/006/MCD12Q1/2000_01_01' not found.